### Analysis of scRNA-seq of Etv2 reprogramming

In [22]:
library(SummarizedExperiment)
library(RColorBrewer)
library(dplyr)
library(parallel)
library(ggplot2)
library(goseq)
library(stringr)

Loading required package: BiasedUrn
Loading required package: geneLenDataBase

Attaching package: ‘geneLenDataBase’

The following object is masked from ‘package:S4Vectors’:

    unfactor




Read the processed scRNA-seq data (~2.1G)
The scRNA-seq dataset was prepared in scRNA_seq_preprocess.Rmd

In [24]:
se <- readRDS(gzcon(url('https://s3.msi.umn.edu/garry_projects/etv2_pioneer/processed_Etv2_scRNAseq.rds')))
se

class: SummarizedExperiment 
dim: 17010 15475 
metadata(1): curves
assays(3): counts normalized_counts scaled_counts
rownames: NULL
rowData names(6): id name ... is_hvg clusters
colnames: NULL
colData names(7): barcodes group ... umap cluster

class: SummarizedExperiment 
dim: 17010 15475 
metadata(1): curves
assays(3): counts normalized_counts scaled_counts
rownames: NULL
rowData names(6): id name ... is_hvg clusters
colnames: NULL
colData names(7): barcodes group ... umap cluster
